In [34]:
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', 500)

In [26]:
datadir = Path("../data/")
etccdi_presentf = datadir / "Climate"/ "ISEA3H09_CCSM4_Y1950_Y2000_ETCCDI_IDW1N10.txt"

# Exploring the ETCCDI climate variables

These variables weere derived by aggregating ETCCDI variables of the CCSM4 grid cells into ISEA3H09. The interpolation method was  IDW that maps the 10 nearest CCSM4 grid cells to one ISEA3H09 hexagon centroid.

## I. Exploring the 1950-2000 data

In [27]:
# dowmloading the data
present = pd.read_csv(etccdi_presentf, sep="\t")
present.sample(5)

In [36]:
# basic statistics, no abnormal min and max values
present.describe(include="all")

,HID,CDD_IDW1N10,CSDI_IDW1N10,CWD_IDW1N10,DTR_IDW1N10,FD_IDW1N10,GSL_IDW1N10,ID_IDW1N10,PRCPTOT_IDW1N10,R10MM_IDW1N10,R1MM_IDW1N10,R20MM_IDW1N10,R95P_IDW1N10,R99P_IDW1N10,RX1DAY_IDW1N10,RX5DAY_IDW1N10,SDII_IDW1N10,SU_IDW1N10,TN10P_IDW1N10,TN90P_IDW1N10,TNN_IDW1N10,TNX_IDW1N10,TR_IDW1N10,TX10P_IDW1N10,TX90P_IDW1N10,TXN_IDW1N10,TXX_IDW1N10,WSDI_IDW1N10
count,196832.00000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000,196832.000000
mean,98416.50000,53.461916,5.921488,19.448150,3.913968,66.152896,283.407494,49.761787,1027.460573,28.711493,154.435523,8.879405,236.388862,75.945661,43.664978,92.993421,5.987150,134.004543,10.170603,11.166394,1.226858,20.310472,159.527339,10.198060,11.146702,4.758275,24.631820,8.267821
std,56820.64843,120.135759,3.414841,16.634862,3.965489,119.556650,136.089634,105.124877,740.698730,26.403323,74.932983,10.030891,174.392344,57.529881,24.547932,59.347809,2.390525,145.094095,0.658983,0.724655,23.936553,10.356050,156.289682,0.653036,0.753927,22.663115,11.696714,5.638105
min,1.00000,4.009007,0.067093,0.042284,0.407698,0.000000,0.000000,0.000000,0.055981,0.000000,0.042284,0.000000,0.000000,0.000000,0.669288,1.662915,0.053741,0.000000,7.797225,8.633949,-74.617955,-30.565328,0.000000,8.091539,8.599853,-72.497892,-25.841064,0.022317
25%,49208.75000,13.973389,3.384982,8.654831,0.669764,0.000000,223.528982,0.000000,487.364333,8.249519,104.463420,1.042032,106.375416,33.203361,23.241466,45.351262,4.036451,0.000000,9.750012,10.688299,-9.019107,16.460489,0.014944,9.801452,10.638720,-4.050492,20.446503,4.176016
50%,98416.50000,23.761283,5.672391,13.472844,1.979585,0.000000,365.000000,0.000000,951.884730,22.361179,155.981161,5.309775,204.332062,63.245950,40.070109,78.636339,5.765742,66.131205,10.141659,11.115011,9.166234,24.193069,113.964472,10.168131,11.054468,13.112167,27.765465,7.481039
75%,147624.25000,53.208218,7.945608,25.749495,6.539862,72.862497,365.000000,12.526608,1349.030337,40.499057,208.386895,13.536889,323.305303,104.938461,61.973293,130.848140,7.720937,289.566001,10.540513,11.598794,20.429288,27.705534,361.674213,10.574305,11.588533,22.460634,30.087498,11.377216
max,196832.00000,11719.647151,22.658614,192.748037,15.882537,365.000000,365.000000,365.000000,6041.701696,197.288133,340.268278,108.359245,1299.631735,399.696169,177.918199,486.991962,22.302361,365.000000,13.330696,15.019223,26.222249,36.102259,365.000000,13.138606,14.961791,27.597581,51.419834,34.264405


In [30]:
# check for missing data - seems to be none
present.isnull().values.any()

False

In [ ]:
# precipitation related, measured in days: CDD, CWD
# precipitation related, measured in days with length threshold: CSDI, WSDI
# precipitation related, measured in mm: 
# temperature related, range of temperature: DTR
# temperature related, measured in days: FD
# temperature related, measured in days with length threshold: GSL